<a href="https://colab.research.google.com/github/caltdreamer/GNN/blob/main/Conditional_Coverage_GraphSage_classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install conditionalconformal

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q torch-geometric
!pip install ogb
!pip install umap-learn

In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GraphSAGE
from torch_geometric.data import NeighborSampler
from torch_geometric.nn import SAGEConv
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import NELL
from torch_geometric.datasets import CitationFull
from torch_geometric.datasets import CoraFull
from torch_geometric.data import DataLoader
from tqdm import tqdm
import torch.optim as optim
import numpy as np
import os.path as osp
import pandas as pd
import numpy as np
import collections
import random
import networkx as nx
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

In [ ]:
dataset = CoraFull(root='./data')
dataset = Planetoid(root='./data', name='Cora',split = 'public')



In [ ]:
data = dataset[0]
print(data)
print(data.x)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = data.x.to(device)
y = data.y.squeeze().to(device)


In [ ]:
random.seed(42)

In [ ]:
#Randomly choose train and calitest set
train_percentage = 0.6
test_percentage = 0.5



num_nodes = data.num_nodes
all_node_indices = list(range(num_nodes))
random.shuffle(all_node_indices)
num_train_nodes = int(train_percentage * num_nodes)
num_testandcali_nodes = num_nodes - num_train_nodes
train_node_indices = all_node_indices[:num_train_nodes]
testandcali_node_indices = all_node_indices[num_train_nodes:]

print(testandcali_node_indices)
test_node_indices = testandcali_node_indices[:int(test_percentage * num_testandcali_nodes)]
cali_node_indices = testandcali_node_indices[int(test_percentage * num_testandcali_nodes):]

print(test_node_indices)

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[train_node_indices] = True
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[train_node_indices] = True

testandcali_mask = torch.zeros(num_nodes,dtype=torch.bool)
testandcali_mask[testandcali_node_indices] = True

test_mask = torch.zeros(num_nodes,dtype=torch.bool)
test_mask[test_node_indices] = True


cali_mask = torch.zeros(num_nodes,dtype=torch.bool)
cali_mask[cali_node_indices] = True




print(train_mask.sum())
print(test_mask.sum())
print(cali_mask.sum())




print(test_node_indices)


In [ ]:
train_loader = NeighborSampler(
    data.edge_index, node_idx=train_mask,
    sizes=[5,5,5], batch_size=64, shuffle=False,
)

In [8]:
class SAGE(torch.nn.Module):
    def __init__(self,in_channels,hidden_channels,out_channels,num_layers = 3):
        super(SAGE,self).__init__()

        self.numlayers = num_layers
        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels,hidden_channels))
        for i in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels,hidden_channels))
        self.convs.append(SAGEConv(hidden_channels,out_channels))
    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
    def forward(self,x,adjs):
        for i ,(edge_index,_,size) in enumerate(adjs):
            xs = []
            x_target = x[:size[1]]#x should be a list prioritizing target nodes
            x = self.convs[i]((x,x_target),edge_index)
            if i != self.numlayers -1:
                x = F.relu(x)
                x = F.dropout(x,p=0.5,training=self.training)
            xs.append(x)
            if i == 0:
                x_all = torch.cat(xs, dim=0)
                layer_1_embeddings = x_all
            elif i == 1:
                x_all = torch.cat(xs, dim=0)
                layer_2_embeddings = x_all
            elif i == 2:
                x_all = torch.cat(xs, dim=0)
                layer_3_embeddings = x_all
        return layer_1_embeddings, layer_2_embeddings, layer_3_embeddings

    def inference(self, x,adjs):
        for i ,(edge_index,_,size) in enumerate(adjs):
            xs = []
            x_target = x[:size[1]]#x should be a list prioritizing target nodes
            x = self.convs[i]((x,x_target),edge_index)
            if i != self.numlayers -1:
                x = F.relu(x)
                x = F.dropout(x,p=0.5,training=self.training)
            xs.append(x)
            if i == 0:
                x_all = torch.cat(xs, dim=0)
                layer_1_embeddings = x_all
            elif i == 1:
                x_all = torch.cat(xs, dim=0)
                layer_2_embeddings = x_all
            elif i == 2:
                x_all = torch.cat(xs, dim=0)
                layer_3_embeddings = x_all
        return layer_1_embeddings, layer_2_embeddings, layer_3_embeddings


In [9]:
model = SAGE(dataset.num_features, 64, dataset.num_classes, num_layers=3)

model.reset_parameters()
#We only input the initializing variables
model = model.to(device)

In [ ]:
print(model)

In [11]:
def train(epoch):
    model.train()
    total_loss = total_correct =0

    for batch_size,n_id,adjs in train_loader:
        adjs = [adj.to(device) for adj in adjs]
        optimizer.zero_grad()
        l1_emb, l2_emb, l3_emb = model(x[n_id], adjs)
        out = l3_emb.log_softmax(dim=-1)
        loss = F.nll_loss(out,y[n_id[:batch_size]])
        loss.backward()
        optimizer.step()

        total_loss += float(loss)
        total_correct += int(out.argmax(dim=-1).eq(y[n_id[:batch_size]]).sum())


    loss = total_loss / len(train_loader)


    approx_acc = total_correct / int(train_mask.sum())

    return loss, approx_acc

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

for epoch in range(1,20):
    loss,acc = train(epoch)
    print(f'Epoch {epoch:02d}, Loss: {loss:.4f}, Approx. Train: {acc:.4f}')

In [13]:
CaliGraph_loader = NeighborSampler(data.edge_index,node_idx = cali_mask
  , sizes = [5,5,5],batch_size = 5000,shuffle=False)

In [14]:
TestGraph_loader  = NeighborSampler(data.edge_index,node_idx = test_mask
  , sizes = [5,5,5],batch_size = 5000,shuffle=False)

In [16]:
#function used to output final layer embedding and predicted labels
def inference_1(subgraph_loader):
    model.eval()
    total_loss = total_correct = 0
    for batch_size,n_id,adjs in subgraph_loader:

        emb1,emb2,emb3 = model(x[n_id],adjs)
        out = emb3.log_softmax(dim=-1)
        y_pred = out.argmax(dim=-1,keepdim =True)



    return emb3, y_pred


In [ ]:
Cali_labels_idx = []

for i,(batch_size,n_id,adjs) in enumerate(CaliGraph_loader):

   print(batch_size)
   print(n_id)
   Cali_labels_idx = n_id
print(Cali_labels_idx)

In [ ]:
#Use this one

emb3_Cali,y_pred_Cali = inference_1(CaliGraph_loader)
emb3_Cali = emb3_Cali.softmax(dim=-1)
#numpy_emb3_Cali = emb3_Cali.detach().cpu().numpy()
#row_sums = np.sum(numpy_emb3_Cali, axis=1).reshape(-1, 1)
#emb3_Cali = numpy_emb3_Cali/row_sums


numpy_y_pred_Cali = y_pred_Cali.detach().cpu().numpy()
numpy_y_labels_Cali = y[cali_mask].detach().cpu().numpy()
numpy_y_labels_Test = y[test_mask].detach().cpu().numpy()
numpy_y_labels_column = np.array(numpy_y_labels_Cali).reshape(-1, 1)
#accuracy of our model for calibration nodes
acc = (numpy_y_pred_Cali == numpy_y_labels_column).sum()/cali_mask.sum()
print(acc)

In [ ]:
alpha = 0.05
n=cali_mask.sum()
m=test_mask.sum()

#Using APS score function
cal_smx_tensor = torch.FloatTensor(emb3_Cali)
cal_smx = cal_smx_tensor.detach().cpu().numpy()
cal_pi = cal_smx.argsort(1)[:, ::-1]
cal_srt = np.take_along_axis(cal_smx,cal_pi,axis=1).cumsum(axis=1)
cal_scores = np.take_along_axis(cal_srt, cal_pi.argsort(axis=1), axis=1)[
    range(n), numpy_y_labels_Cali
]

qhat = np.quantile(
    cal_scores, np.ceil((n + 1) * (1 - alpha)) / n, interpolation="higher"
)

In [ ]:
#Use this one


emb3_Test,y_pred_Test = inference_1(TestGraph_loader)
emb3_Test = emb3_Test.softmax(dim=-1)
numpy_emb3_Test = emb3_Test.detach().cpu().numpy()
test_smx = numpy_emb3_Test
#row_sums = np.sum(numpy_emb3_Test, axis=1).reshape(-1, 1)
#test_smx = numpy_emb3_Test/row_sums

test_pi = test_smx.argsort(1)[:, ::-1]
test_srt = np.take_along_axis(test_smx,test_pi,axis=1).cumsum(axis=1)

test_scores = np.take_along_axis(test_srt, test_pi.argsort(axis=1), axis=1)[
    range(m), numpy_y_labels_Test
]

qhat = np.quantile(
    test_scores, np.ceil((m + 1) * (1 - alpha)) / m, interpolation="higher")

prediction_sets = np.take_along_axis(test_srt <= qhat, test_pi.argsort(axis=1), axis=1)

In [ ]:
empirical = prediction_sets[
    np.arange(prediction_sets.shape[0]), numpy_y_labels_Test
].mean()

print(f"The empirical coverage is: {empirical}")

Next we cosnider group condition covergae by self-implementing the optimization probelm, which turns to be not efficient

In [ ]:
#matrix_1 represents class matrix

matrix_1 = np.zeros((test_mask.sum(), dataset.num_classes), dtype=int)

for i in range(dataset.num_classes):
    matrix_1[:, i] = i


print(matrix_1)



#for each column we compute S(Xi,y) for all possible y categories*****because we will use these in optimization problem
test_scores_foreachclass=np.zeros((test_mask.sum(), dataset.num_classes))


for i in range(dataset.num_classes):
    test_scores = np.take_along_axis(test_srt, test_pi.argsort(axis=1), axis=1)[
    range(m), matrix_1[:,i]]
    test_scores_foreachclass[:,i]=test_scores






In [ ]:
#d represents how many groups we're considering
#simulation only
d = 5

matrix = np.zeros((n + m, d), dtype=int)
for i in range(n + m):
    random_col = np.random.randint(0, 5)
    matrix[i, random_col] = 1



print(matrix)

In [ ]:


import cvxpy as cp
import numpy as np

scores_Cali = cal_scores
scores_Test = test_scores
column_vector_scores_Cali = scores_Cali.reshape(-1, 1)
column_vector_scores_Test = scores_Test.reshape(-1, 1)

column_vector_scores_combined = np.concatenate(
    (column_vector_scores_Cali, column_vector_scores_Test), axis=0
)


#Define the feature vector for calibration and test nodes
X_Cali = x[cali_mask]
X_Test = x[test_mask]
y_Cali = y[cali_mask]
y_Test = y[test_mask]



#list of beta matrix of cardinality test_mask.sum()
#for each matrix inside this list, we have the beta value each row represents argmin with the input S(Xn+i-1,y) for different y
Beta_matrix = []




In [ ]:
Beta_matrix = []
for i in range(test_mask.sum()):
  beta_Xi = np.zeros((dataset.num_classes,d))
  for j in range(dataset.num_classes):



    beta = cp.Variable(d)
    def ell_alpha(y, z):
        return cp.maximum((y - z) * alpha, (z - y) * (1 - alpha))
    objective = cp.Minimize(
        (sum([ell_alpha(matrix[i] @ beta, column_vector_scores_combined[i]) for i in range(n)])
        + ell_alpha(matrix[n] @ beta, test_scores_foreachclass[i,j])
    )/(n+1))

    problem = cp.Problem(objective)
    problem.solve()

    optimal_beta = beta.value
    beta_Xi[j,:]=optimal_beta.reshape(1, -1)




  Beta_matrix.append(beta_Xi)
  print(i)







In [ ]:
#suppose we have already got the Beta_matrix, the next task is to output the prediction interval
prediction_set = np.zeros((test_mask.sum(),dataset.num_classes))
for i in range(test_mask.sum()):
  prediction = []
  beta_matrix = Beta_matrix[i]
  for j in range(dataset.num_classes):
    if(test_scores_foreachclass[i,j]<= matrix[n+j].reshape(1,-1)@(beta_matrix[j]) ):
     prediction.append(1)

    else:
     prediction.append(0)

  prediction_set[i,:]=np.array(prediction).reshape(1, -1)







In [ ]:
#check empirical covergae(marginal first)
numpy_y_Test = y_Test.detach().cpu().numpy()
total_correctness = 0
for i in range(test_mask.sum()):
  for j in range(dataset.num_classes):
    if((prediction_set[i,j] == 1 )& (numpy_y_Test[i]==j)):
       total_correctness +=1


empirical_coverage = total_correctness/(test_mask.sum())
print(f"The empirical coverage is: {empirical_coverage}")

print(f"The empirical miscoverage is: {1-empirical_coverage}")


In [ ]:
matrix_test = matrix[n:n+m]
group_coverage_listgroup =[]

#Hopefully we have achieved group conditional coverage and lets check over d groups
for i in range(d):
  total_correct = 0
  num_data = 0
  for j in range(test_mask.sum()):
    if(matrix_test[j,i] == 1):
      num_data+=1
      for k in range(dataset.num_classes):
          if(prediction_set[j,k] == 1 ):
            if(numpy_y_Test[j]==k):
             total_correct +=1

  group_coverage = total_correct/num_data
  group_coverage_listgroup.append(group_coverage)


for i in range(d):
  print(f"The empirical group coverage for the {i+1 } th group is: {group_coverage_listgroup[i]}")